In [1]:
import json
import pandas as pd
import numpy as np
from tqdm.std import tqdm
import re
import ast

In [2]:
result_path = "/home/yw937/FinBen/results/xbrl_nen/deepseek-chat/samples_XBRL_NEN_2025-05-09T18-49-38.889449.jsonl"

In [3]:
def parse_result(json_line):
    content = json_line["doc"].get("query")
    
    answer = json_line.get("target")
    answer = json.loads(answer)
    
    response = json_line.get("filtered_resps")[0]

    return content, answer, response

In [4]:
def extract_queryAndcandidates(content):
    input_query_match = re.search(r"Input Query:\s*(.*?)\s*Candidate Tags:", content, re.DOTALL)
    input_query = input_query_match.group(1).strip()
    
    candidate_tags_match = re.search(r"Candidate Tags:\s*(\[[\s\S]*?\])\s*Answer:", content, re.DOTALL)
    candidate_tags = candidate_tags_match.group(1).strip()
    candidate_tags = ast.literal_eval(candidate_tags)
    
    return input_query, candidate_tags

In [5]:
def normalize_response(text_str):
    pattern = r"\{\"result\"\s*:\s*(.*?)\}"

    code_block_match = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", text_str, flags=re.DOTALL)
    if code_block_match:
        text_str = code_block_match.group(1).strip()
    else:
        json_match = re.search(pattern, text_str)
        if json_match:
            text_str = json_match.group(1)
            if text_str.startswith('{') and text_str.endswith('}'):
                text_str = text_str.replace("\n", "").strip()
            else:
                # text_str = text_str.strip('"')
                text_str = f"""{{"result": {text_str}}}"""
        else:
            text_str = """{{"result": "None"}}"""

    text_str = text_str.replace("\n", "").strip()
    return text_str.strip()

In [6]:
def calculate_acc(ground_truth, predict_answer):
    correct = [g == p for g, p in zip(ground_truth, predict_answer)]
    accuracy = sum(correct) / len(ground_truth)

    return round(accuracy,4)

In [7]:
ground_truth = []
predict_answer = []
orginal_answer = []
errors = 0
with open(result_path, "r") as f:
    for line in tqdm(f):
        json_line = json.loads(line)
        
        content, answer, response = parse_result(json_line)
        query, candidates = extract_queryAndcandidates(content)
        orginal_answer.append(response)

        response = normalize_response(response)
        # print(response)
        try:
            response = json.loads(response)
            # print(response)
            response  = response.get("result")
            # orginal_answer.append(response)
            if response == "None":
                response = candidates[0]
                errors += 1
        except:
            # orginal_answer.append(response)
            response = candidates[0]
            errors += 1

        answer = answer.get("result")

        ground_truth.append(answer)
        predict_answer.append(response)

        

52572it [00:31, 1689.99it/s]


In [8]:
calculate_acc(ground_truth, predict_answer)

0.1715